In [6]:
from PIL import Image
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import requests

ModuleNotFoundError: No module named 'PIL'

In [9]:
df = pd.read_csv('..\data\csv\data.csv')
display(df.head())
df.count()
sampled_df = df.sample(frac=0.001, random_state=42)
display(sampled_df.head())
sampled_df.count()

,image_url,class
0,https://raw.githubusercontent.com/spMohanty/Pl...,Tomato___Bacterial_spot
1,https://raw.githubusercontent.com/spMohanty/Pl...,Tomato___Late_blight
2,https://raw.githubusercontent.com/spMohanty/Pl...,Tomato___Leaf_Mold
3,https://raw.githubusercontent.com/spMohanty/Pl...,Tomato___Spider_mites Two-spotted_spider_mite
4,https://raw.githubusercontent.com/spMohanty/Pl...,Tomato___Late_blight


,image_url,class
1512,https://raw.githubusercontent.com/spMohanty/Pl...,Tomato___Septoria_leaf_spot
5982,https://raw.githubusercontent.com/spMohanty/Pl...,Tomato___Leaf_Mold
7106,https://raw.githubusercontent.com/spMohanty/Pl...,Tomato___Target_Spot
960,https://raw.githubusercontent.com/spMohanty/Pl...,Tomato___Septoria_leaf_spot
2771,https://raw.githubusercontent.com/spMohanty/Pl...,Tomato___Septoria_leaf_spot


image_url    9
class        9
dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

X = sampled_df['image_url']
y = sampled_df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

print("Training set size:", len(X_train))
print("Testing set size:", len(X_test))

Training set size: 7
Testing set size: 2


In [16]:
import requests
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd

import requests
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans

def process_image_from_url(url, label, show=False):
    # Load GitHub token for authentication
    with open(r"G:/My Drive/Codes/Tokens/github_token.txt", "r") as f:
        token = f.read().strip()
    headers = {"Authorization": f"token {token}"}

    try:
        # Download image
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status()
        img_array = np.asarray(bytearray(response.content), dtype=np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        if img is None:
            print(f"Failed to decode image: {url}")
            return None, None

        # Convert to RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Resize for faster processing
        img_rgb = cv2.resize(img_rgb, (256, 256))

        # Preprocessing: median blur + histogram equalization in HSV
        img_rgb = cv2.medianBlur(img_rgb, 5)
        img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
        img_hsv[:, :, 2] = cv2.equalizeHist(img_hsv[:, :, 2])
        img_rgb = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)

        # Flatten pixels for clustering
        pixels = img_rgb.reshape(-1, 3)

        # MiniBatchKMeans for faster clustering
        k = 5
        kmeans = MiniBatchKMeans(n_clusters=k, random_state=42, batch_size=1024)
        labels = kmeans.fit_predict(pixels)
        segmented_img = kmeans.cluster_centers_[labels].reshape(img_rgb.shape).astype(np.uint8)

        # Extract compact statistical features per cluster
        cluster_features = []
        for i in range(k):
            cluster_pixels = pixels[labels == i]
            if len(cluster_pixels) > 0:  # avoid empty cluster
                cluster_mean = np.mean(cluster_pixels, axis=0)
                cluster_var = np.var(cluster_pixels, axis=0)
            else:  # empty cluster, just pad with zeros
                cluster_mean = [0, 0, 0]
                cluster_var = [0, 0, 0]
            cluster_features.extend([*cluster_mean, *cluster_var])

        feature_vector = np.array(cluster_features, dtype=np.float32)

        # Optional visualization
        if show:
            fig, axes = plt.subplots(1, 2, figsize=(10, 5))
            axes[0].imshow(img_rgb)
            axes[0].set_title("Preprocessed Image")
            axes[0].axis('off')
            axes[1].imshow(segmented_img)
            axes[1].set_title("Segmented Image (k-means)")
            axes[1].axis('off')
            plt.show()

        return feature_vector, label

    except requests.exceptions.RequestException as e:
        print(f"Error downloading image from {url}: {e}")
        return None, None



processed_images_train = []

for url, label in zip(X_train, y_train):
    features, label = process_image_from_url(url,label)
    if features is not None:
        processed_images_train.append((features, label))

print(f"Number of processed images: {len(processed_images_train)}")


processed_images_test = []

for url, label in zip(X_test, y_test):
    features, label = process_image_from_url(url,label)
    if features is not None:
        processed_images_test.append((features, label))

print(f"Number of processed images: {len(processed_images_test)}")

# Save processed data to CSV
if processed_images_train:
    train_df = pd.DataFrame([item[0].tolist() for item in processed_images_train])
    train_df['label'] = [item[1] for item in processed_images_train]
    train_df.to_csv('..\data\csv\processed_images_train.csv', index=False)

if processed_images_test:
    test_df = pd.DataFrame([item[0].tolist() for item in processed_images_test])
    test_df['label'] = [item[1] for item in processed_images_test]
    test_df.to_csv('..\data\csv\processed_images_test.csv', index=False)

print("Processed data saved to processed_images_train.csv and processed_images_test.csv")

Number of processed images: 7
Number of processed images: 2
Processed data saved to processed_images_train.csv and processed_images_test.csv


In [13]:
import pandas as pd
import numpy as np

# Read CSV
train_df = pd.read_csv('processed_images_train.csv')
test_df = pd.read_csv('processed_images_test.csv')

# Convert DataFrame rows to list of tuples
# Assumes the CSV has columns: 'feature_vector', 'label'
def df_to_list_of_arrays(df):
    processed_list = []
    for _, row in df.iterrows():
        # Convert string vector to np.array
        arr = np.fromstring(row['features'].strip('[]'), sep=' ')
        processed_list.append((arr, row['label']))
    return processed_list

processed_images_train = df_to_list_of_arrays(train_df)
processed_images_test = df_to_list_of_arrays(test_df)

print("Data read from CSV and converted back to list format.")
print(f"Number of items in processed_images_train: {len(processed_images_train)}")
print(f"Number of items in processed_images_test: {len(processed_images_test)}")

# Example
print(processed_images_train[0])
print(processed_images_test[0])


KeyError: 'features'

In [14]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Separate features and labels from the processed data
X_processed_train = np.array([item[0] for item in processed_images_train])
y_processed_train = np.array([item[1] for item in processed_images_train])

# Prepare test data
X_processed_test = np.array([item[0] for item in processed_images_test])
y_processed_test = np.array([item[1] for item in processed_images_test])

# Dictionary to store results
results = {}

# Initialize and train SVM
svm_model = SVC()
svm_model.fit(X_processed_train, y_processed_train)
y_pred_svm = svm_model.predict(X_processed_test)
results['SVM'] = {
    'Accuracy': accuracy_score(y_processed_test, y_pred_svm),
    'Classification Report': classification_report(y_processed_test, y_pred_svm, output_dict=True)
}

# Initialize and train Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(X_processed_train, y_processed_train)
y_pred_dt = dt_model.predict(X_processed_test)
results['Decision Tree'] = {
    'Accuracy': accuracy_score(y_processed_test, y_pred_dt),
    'Classification Report': classification_report(y_processed_test, y_pred_dt, output_dict=True)
}

# Initialize and train Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_processed_train, y_processed_train)
y_pred_rf = rf_model.predict(X_processed_test)
results['Random Forest'] = {
    'Accuracy': accuracy_score(y_processed_test, y_pred_rf),
    'Classification Report': classification_report(y_processed_test, y_pred_rf, output_dict=True)
}

# Initialize and train KNN
knn_model = KNeighborsClassifier()
knn_model.fit(X_processed_train, y_processed_train)
y_pred_knn = knn_model.predict(X_processed_test)
results['KNN'] = {
    'Accuracy': accuracy_score(y_processed_test, y_pred_knn),
    'Classification Report': classification_report(y_processed_test, y_pred_knn, output_dict=True)
}

# Display results in a table
accuracy_data = {model: results[model]['Accuracy'] for model in results}
accuracy_df = pd.DataFrame.from_dict(accuracy_data, orient='index', columns=['Accuracy'])
print("Model Performance Summary:")
display(accuracy_df)

print("\nClassification Reports:")
for model, metrics in results.items():
    print(f"\n--- {model} ---")
    report_df = pd.DataFrame(metrics['Classification Report']).transpose()
    display(report_df)

Model Performance Summary:


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.

,Accuracy
SVM,0.0
Decision Tree,0.0
Random Forest,0.0
KNN,0.0



Classification Reports:

--- SVM ---


,precision,recall,f1-score,support
Tomato___Leaf_Mold,0.0,0.0,0.0,1.0
Tomato___Septoria_leaf_spot,0.0,0.0,0.0,0.0
Tomato___Spider_mites Two-spotted_spider_mite,0.0,0.0,0.0,1.0
accuracy,0.0,0.0,0.0,0.0
macro avg,0.0,0.0,0.0,2.0
weighted avg,0.0,0.0,0.0,2.0


...

,precision,recall,f1-score,support
Tomato___Bacterial_spot,0.0,0.0,0.0,0.0
Tomato___Leaf_Mold,0.0,0.0,0.0,1.0
Tomato___Spider_mites Two-spotted_spider_mite,0.0,0.0,0.0,1.0
Tomato___Target_Spot,0.0,0.0,0.0,0.0
accuracy,0.0,0.0,0.0,0.0
macro avg,0.0,0.0,0.0,2.0
weighted avg,0.0,0.0,0.0,2.0



--- Random Forest ---


,precision,recall,f1-score,support
Tomato___Leaf_Mold,0.0,0.0,0.0,1.0
Tomato___Septoria_leaf_spot,0.0,0.0,0.0,0.0
Tomato___Spider_mites Two-spotted_spider_mite,0.0,0.0,0.0,1.0
Tomato___Target_Spot,0.0,0.0,0.0,0.0
accuracy,0.0,0.0,0.0,0.0
macro avg,0.0,0.0,0.0,2.0
weighted avg,0.0,0.0,0.0,2.0



--- KNN ---


,precision,recall,f1-score,support
Tomato___Leaf_Mold,0.0,0.0,0.0,1.0
Tomato___Septoria_leaf_spot,0.0,0.0,0.0,0.0
Tomato___Spider_mites Two-spotted_spider_mite,0.0,0.0,0.0,1.0
accuracy,0.0,0.0,0.0,0.0
macro avg,0.0,0.0,0.0,2.0
weighted avg,0.0,0.0,0.0,2.0


In [15]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np

# Assuming X_processed_train, y_processed_train, X_processed_test, and y_processed_test are already defined from previous steps

# Initialize and train Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_processed_train, y_processed_train)
y_pred_nb = nb_model.predict(X_processed_test)

# Evaluate Naive Bayes
nb_accuracy = accuracy_score(y_processed_test, y_pred_nb)
nb_report = classification_report(y_processed_test, y_pred_nb, output_dict=True)

print("Naive Bayes Accuracy:", nb_accuracy)
print("Naive Bayes Classification Report:\n")
display(pd.DataFrame(nb_report).transpose())

# You can optionally add Naive Bayes results to the 'results' dictionary if it exists
# from the previous cell (zFbG8ZI5wACc)
# if 'results' in locals():
#     results['Naive Bayes'] = {
#         'Accuracy': nb_accuracy,
#         'Classification Report': nb_report
#     }
#     # Re-display the combined results table if you added Naive Bayes
#     # accuracy_data = {model: results[model]['Accuracy'] for model in results}
#     # accuracy_df = pd.DataFrame.from_dict(accuracy_data, orient='index', columns=['Accuracy'])
#     # print("\nUpdated Model Performance Summary:")
#     # display(accuracy_df)

Naive Bayes Accuracy: 0.0
Naive Bayes Classification Report:



C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.

,precision,recall,f1-score,support
Tomato___Leaf_Mold,0.0,0.0,0.0,1.0
Tomato___Septoria_leaf_spot,0.0,0.0,0.0,0.0
Tomato___Spider_mites Two-spotted_spider_mite,0.0,0.0,0.0,1.0
accuracy,0.0,0.0,0.0,0.0
macro avg,0.0,0.0,0.0,2.0
weighted avg,0.0,0.0,0.0,2.0
